In [1]:
import torch
import torch.nn.functional as F
import torchtext
import time
import os

In [2]:
class Network(torch.nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_size, num_outputs, num_layers=2):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, emb_dim)
        # Bidirectional LSTM
        self.rnn = torch.nn.LSTM(emb_dim,
                                 hidden_size,
                                 num_layers=num_layers,
                                 dropout=0.2,
                                 bidirectional=True)
        self.fc = torch.nn.Linear(hidden_size * 2, num_outputs)
        self.softmax = torch.nn.LogSoftmax(dim=-1)
        
    def forward(self, inputs):
        embs = self.embedding(inputs)
        output, _ = self.rnn(embs)
        output = self.fc(output[-1])
        return self.softmax(output)

In [3]:
# import torch
# import torchtext

def LoadTSV(file_path, columns, skip_header=True):
    return torchtext.data.TabularDataset(file_path, 'TSV', columns, skip_header=skip_header)

In [4]:
LABEL = torchtext.data.Field(sequential=False, use_vocab=False, dtype=torch.long)
TEXT = torchtext.data.Field(fix_length=50, use_vocab=True, lower=True)

train_columns = [
    ('PhraseId', None),
    ('SentenceId', None),
    ('Phrase', TEXT),
    ('Sentiment', LABEL)
]

test_columns = [
    ('PhraseId', None),
    ('SentenceId', None),
    ('Phrase', TEXT)
]

In [5]:
train = LoadTSV('./dataset/train.tsv/train.tsv', train_columns)
test = LoadTSV('./dataset/test.tsv/test.tsv', test_columns)

In [6]:
TEXT.build_vocab(train, vectors=torchtext.vocab.GloVe(name='6B', dim=300), max_size=50000)
LABEL.build_vocab(train)

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
train_iter, test_iter = torchtext.data.BucketIterator.splits((train, test), batch_size=32, device=device)

train_iter.repeat = False
test_iter.repeat = False

In [10]:
vocab_size = len(TEXT.vocab)
emb_dim = 300
hidden_size = 300
num_outputs = 5
num_layers = 2

model = Network(vocab_size, emb_dim, hidden_size, num_outputs, num_layers=num_layers)
model.embedding.weight.data = TEXT.vocab.vectors
model.embedding.weight.require_grad = False

if torch.cuda.is_available():
    model = model.cuda()

epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

test_loss, test_accuracy = [], []

def training(model, dataset, optimizer, scheduler, epochs=10):
    model.train()
    training_loss, train_accuracy = [], []
    dataset_size = len(dataset.dataset)
    for epoch in range(epochs):
        epoch_begin = time.time()
        epoch_loss = 0.0
        epoch_corrects = 0
        print(f'------------- Epoch {epoch + 1} -------------')
        for batch in dataset:
            text, labels = batch.Phrase, batch.Sentiment
            if torch.cuda.is_available():
                text, labels = text.cuda(), labels.cuda()
            optimizer.zero_grad()
            output = model(text)
            loss = F.nll_loss(output, labels, reduction='sum')
            _, preds = torch.max(output, dim=1)
            epoch_loss += loss.data.detach().item()
            epoch_corrects += preds.eq(labels.data.view_as(preds)).sum()
            
            loss.backward()
            optimizer.step()
        print(f'Loss / Accuracy : {epoch_loss / dataset_size :.4f} / {100. * epoch_corrects / dataset_size :.4f}% === {time.time() - epoch_begin}')
        scheduler.step()
                
training(model, train_iter, optimizer, scheduler, epochs=epochs)

------------- Epoch 1 -------------
Loss / Accuracy : 0.9379 / 61.9191% === 103.4029757976532
------------- Epoch 2 -------------
Loss / Accuracy : 0.7307 / 69.7571% === 104.28602504730225
------------- Epoch 3 -------------
Loss / Accuracy : 0.6605 / 72.5759% === 105.09877872467041
------------- Epoch 4 -------------
Loss / Accuracy : 0.6086 / 74.7341% === 104.50084233283997
------------- Epoch 5 -------------
Loss / Accuracy : 0.5600 / 76.6526% === 103.95793652534485
------------- Epoch 6 -------------
Loss / Accuracy : 0.4552 / 81.2982% === 104.29112648963928
------------- Epoch 7 -------------


KeyboardInterrupt: 